# Variant calling

    Going to use bcftools concensus calling (see https://github.com/tparchman/T.podura/blob/master/analysis_workflow_varcal.md).   
    Bam files have been split by chromosome (9) already in 3mapping

In [8]:
!source activate py36

/usr/bin/sh: line 0: source: activate: file not found


In [9]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
from os import environ
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
#import Levenshtein as lv

In [10]:
root = "/data/gpfs/assoc/denovo/tfaske/sagebrush"

In [11]:
snp_dir = os.path.join(root,'SNPcall')

In [12]:
bwa_dir = os.path.join(snp_dir,'bwa')

In [13]:
cd $bwa_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/bwa


### Assembly

https://www.ncbi.nlm.nih.gov/assembly/GCA_023558565.1/#/def

Full chromosome assembly, extracted only chromosomes (9) 

In [14]:
assembly = "/data/gpfs/assoc/denovo/tfaske/sagebrush/assembly/chromosome_assembly/GCA_023558565.1_ASM2355856v1/GCA_023558565.1_ASM2355856v1_genomic.fna"
assert assembly

#assembly indexed with bwa: 
# bwa index -a bwtsw -p chromosome_assembly chromosome_assembly.fna
# index_bwa.sh 

In [29]:
mapped_bam = []
files = !find . -type f -name '*mapped.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    mapped_bam.append(x)
mapped_bam = sorted(mapped_bam)

In [30]:
len(mapped_bam), mapped_bam[0]

(708,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/bwa/split_bam/AT2_BB_10_mapped.bam')

## split bam files by chromosome

In [15]:
cd $bwa_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/bwa


In [35]:
!mkdir split_bam

mkdir: cannot create directory ‘split_bam’: File exists


In [16]:
split_dir = os.path.join(bwa_dir,"split_bam")
assert(split_dir)

In [ ]:
#mv mapped bam to spilt_bam
for mb in mapped_bam:
    !mv $mb $split_dir

In [17]:
cd $split_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/bwa/split_bam


In [18]:
mapped_bam = []
files = !find . -type f -name '*mapped.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    mapped_bam.append(x)
mapped_bam = sorted(mapped_bam)

In [19]:
len(mapped_bam), mapped_bam[0]

(708,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/bwa/split_bam/AT2_BB_10_mapped.bam')

In [ ]:
for mb in mapped_bam:                                     
    cmd = "bamtools split -in %s -reference" % (mb)
    !$cmd

In [40]:
!ls -l *REF_CM042338.1.bam | wc -l 

708


# Call snps by chromosome

    Using bcftools modeled after https://github.com/tparchman/T.podura/blob/master/analysis_workflow_varcal.md
    
    Need to make one text file that lists bam files one per line, one file per chromosome

In [20]:
cd $split_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/bwa/split_bam


In [21]:
chrom_bam = []
chrom_files = !find . -type f -name '*mapped.REF*bam'
chrom_files = [os.path.abspath(x) for x in chrom_files if 'bam' in x]
for x in chrom_files:
    chrom_bam.append(x)
chrom_bam = sorted(chrom_bam)

In [22]:
len(chrom_bam),chrom_bam[0]

(6372,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/bwa/split_bam/AT2_BB_10_mapped.REF_CM042338.1.bam')

In [23]:
chroms = sorted(list(set([b.split('REF_')[1].split('.bam')[0] for b in chrom_bam])))

In [24]:
len(chroms),chroms

(9,
 ['CM042338.1',
  'CM042339.1',
  'CM042340.1',
  'CM042341.1',
  'CM042342.1',
  'CM042343.1',
  'CM042344.1',
  'CM042345.1',
  'CM042346.1'])

In [25]:
cd $snp_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall


In [87]:
!mkdir vcf

In [26]:
vcf_dir = os.path.join(snp_dir,"vcf")
assert(vcf_dir)

In [27]:
cd $vcf_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf


#### make bam file list per chrom

In [28]:
for i in range(0,len(chroms)):
    c_files = [c for c in chrom_files if chroms[i] in c]
    c_files = sorted(c_files)
    outfile = 'chr' + str(i + 1) + '_bam.txt'
    with open(outfile, "w") as o:
        for c in c_files:
            o.write("%s\n" % (c))

#### call snps for bcftools 

In [29]:
cd $vcf_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf


In [30]:
chrom_list = ['chr' + str(l) for l in list(range(1,10))]
chrom_list

['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9']

In [33]:
### select options for slurm submission
#account = 'cpu-s1-bionres-0'
#partition = 'cpu-s1-bionres-0'
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
jobname = 'bcftools'
time = '1-00:00:00' #time limit 1 day
cpus = 1
mem_cpu = 24000
email = 'tfaske@nevada.unr.edu'

In [36]:
def write_bcftools_sh(account,partition,time,jobname,cpus,mem_cpu,email,chr,assembly):
    for i in range(0,len(chr)):
        with open("run_bcftools_%s.sh" % (chr[i]), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_bcftools
#SBATCH --output output_bcftools_%s.txt
#SBATCH --mail-type=FAIL,END
#SBATCH --mail-user=%s \n\n
    
bcftools mpileup -a DP,AD,INFO/AD -C 50 -d 250 -f %s -q 30 -Q 20 -I -b %s_bam.txt | bcftools call -v -c -f GQ -O z -o %s.vcf.gz
 \n""" % (account,partition,time,int(cpus),int(mem_cpu),chr[i],chr[i],email,assembly,chr[i],chr[i]))

In [37]:
write_bcftools_sh(account,partition,time,jobname,cpus,mem_cpu,email,chrom_list,assembly)

#### finds all bcftools slurm scripts and writes bash script to sbatch them

In [38]:
shbcf_files = []
files = !find . -name 'run_bcftools*'
files = [os.path.abspath(x) for x in files]
for x in files:
        shbcf_files.append(x)
shbcf_files = sorted(shbcf_files)

In [39]:
len(shbcf_files),shbcf_files[0]

(9,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/run_bcftools_chr1.sh')

In [40]:
def write_bash_bcf_sh(sh_files):
    with open("run_bash_bcftools.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))    

In [41]:
write_bash_bcf_sh(shbcf_files)

### concatnate vcf files

In [50]:
cd $vcf_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf


In [54]:
vcfs_gz = !ls *vcf.gz | grep -v tbi | grep -v reorder | grep -v sorted
vcfs_gz = [os.path.abspath(x) for x in vcfs_gz]

In [55]:
len(vcfs_gz),vcfs_gz

(9,
 ['/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/chr1.vcf.gz',
  '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/chr2.vcf.gz',
  '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/chr3.vcf.gz',
  '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/chr4.vcf.gz',
  '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/chr5.vcf.gz',
  '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/chr6.vcf.gz',
  '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/chr7.vcf.gz',
  '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/chr8.vcf.gz',
  '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/chr9.vcf.gz'])

In [56]:
with open("run_concat_vcf.sh", "w") as o:
    o.write("bcftools concat --threads 50 -Oz -o ARTR_chr.vcf.gz {}\n".format(" ".join(vcfs_gz)))

## EXTRA TEST 

I messed up initial vcf files but still have half in dir: /data/gpfs/home/tfaske/d/sagebrush/SNPcall/vcf/vcf_part

Reorder and filter and run EBG to get started 

## reorder vcf file

messed up bam file ordering and need to reorder

    Ran in terminal: 
    - cp chr1.vcf.gz index.vcf.gz
    - gunzip index.vcf.gz
    - bcftools query -l index.vcf | sort > sample_index.txt 
    - gunzip *vcf.gz
    - bcftools view -S samples.txt input.vcf > output.vcf

In [42]:
vcf_part = os.path.join(vcf_dir,"vcf_part")
assert(vcf_part)

In [57]:
cd $vcf_part

/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/vcf_part


In [44]:
vcfs_gz = !ls *vcf.gz | grep -v tbi | grep -v sorted
vcfs_gz = [os.path.abspath(x) for x in vcfs_gz]

In [45]:
len(vcfs_gz),vcfs_gz[0]

(5,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/vcf_part/chr5.vcf.gz')

In [46]:
chrom_list = ['chr' + str(l) for l in list(range(5,10))]
chrom_list

['chr5', 'chr6', 'chr7', 'chr8', 'chr9']

In [47]:
for c in chrom_list:
    vcf = c + '.vcf'
    vcf_out = c + '.reorder.vcf'
    !bcftools view -S sample_index.txt $vcf > $vcf_out

In [58]:
for c in chrom_list:
    vcf = c + '.reorder.vcf'
    vcf_gz = vcf + '.gz'
    !bgzip -c {vcf} > {vcf_gz}
    !tabix {vcf_gz}

### concatnate vcf files

In [59]:
cd $vcf_part

/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/vcf_part


In [60]:
vcfs_gz = !ls *reorder.vcf.gz | grep -v tbi | grep -v sorted
vcfs_gz = [os.path.abspath(x) for x in vcfs_gz]

In [61]:
len(vcfs_gz),vcfs_gz[0]

(5,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/vcf/vcf_part/chr5.reorder.vcf.gz')

In [62]:
with open("run_concat_vcf.sh", "w") as o:
    o.write("bcftools concat --threads 50 -Oz -o ARTR_part.vcf.gz {}\n".format(" ".join(vcfs_gz)))